In [1]:
## 好大夫 https://www.haodf.com
## 爬取问诊信息
## 医生、病例、建议、对话

In [2]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import urllib
import re
import time
import json
from IPython.display import clear_output
import os,sys

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()
import chardet

In [3]:
outpath = '../Data/webpage/'
datapath = '../Data/webpage/'

In [4]:
#driver输入参数
options=webdriver.ChromeOptions()
out_path='/Users/yuchanghong/Desktop/haodf/Data/webpage/'
prefs={'profile.default_content_settings.popups': 0, 'download.default_directory': out_path}
options.add_experimental_option('prefs', prefs)

In [5]:
def get_html_files(directory):
    # 获取目录下所有文件
    all_files = os.listdir(directory)

    # 使用正则表达式匹配以数字开头的HTML文件
    pattern = re.compile(r'^\d.*\.html$')
    html_files = [file for file in all_files if pattern.match(file)]
    html_files_without_extension = [os.path.splitext(file)[0] for file in html_files]


    return html_files_without_extension

In [6]:
def save_dataframe(df, file, outpath,suffix):
    # 创建最终的文件名
    filename = f"{file}_{suffix}"

    # 存为 Excel 文件（使用 utf-16 编码）
    excel_filepath = os.path.join(outpath, f"{filename}.xlsx")
    df.to_excel(excel_filepath, index=False, encoding='utf-16')

    # 存为 Python 二进制文件（Pickle 格式，也使用 utf-16 编码）
    pickle_filepath = os.path.join(outpath, f"{filename}.pkl")
    df.to_pickle(pickle_filepath, protocol=5)  # protocol=5 表示使用最高的 Pickle 协议版本
    
    #print(excel_filepath, 'saved.')

## 爬取网页链接

In [11]:
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)

/var/folders/r7/p1s2dy_d55zbvx8s9j4nd_x00000gn/T/ipykernel_1520/1871793919.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
/var/folders/r7/p1s2dy_d55zbvx8s9j4nd_x00000gn/T/ipykernel_1520/1871793919.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)


In [12]:
content_prefix = "https://www.haodf.com/bingcheng/list.html?p="

In [1]:
for i in range(100):
    content_num = str(i+1)
    content = content_prefix + content_num
    
    link=content
    driver.get(link)
    time.sleep(2)
    
    html_content = driver.page_source
    fl_name = "content_" + content_num
    with open(f'{outpath}/{fl_name}.html', 'w', encoding='utf-16') as file:
        file.write(html_content)  
print("content page saved!")

NameError: name 'content_prefix' is not defined

In [ ]:
l = []

In [ ]:
for i in range(100):
    content_num = str(i+1)
    fl_name = "content_" + content_num
    html_file_path = f'{outpath}/{fl_name}.html'
    with open(html_file_path, 'r', encoding='utf-16') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, 'html.parser')
    links = soup.find_all('span', class_='fl') 
    # Extract and print the URLs
    for link in links:
        a_tag = link.find('a')
        if a_tag:
            url = a_tag.get('href')
            #print(url)
            l.append(url)
    if i % 20 == 0:
        print(''.join(['第',str(i+1),"个网页中的链接已储存，现在共有",str(len(l)),'个链接。']))

In [ ]:
# Save the list to a NumPy file
np.save(f'{outpath}/content.npy', l)

In [ ]:
driver.quit()

## 下载网页

In [7]:
l = np.load(f'{outpath}/content.npy')
print(len(l))
print(l[:10])

9000
['https://www.haodf.com/bingcheng/8897290087.html'
 'https://www.haodf.com/bingcheng/8897289968.html'
 'https://www.haodf.com/bingcheng/8897289927.html'
 'https://www.haodf.com/bingcheng/8897289925.html'
 'https://www.haodf.com/bingcheng/8897289914.html'
 'https://www.haodf.com/bingcheng/8897289904.html'
 'https://www.haodf.com/bingcheng/8897289896.html'
 'https://www.haodf.com/bingcheng/8897289892.html'
 'https://www.haodf.com/bingcheng/8897289879.html'
 'https://www.haodf.com/bingcheng/8897289875.html']


In [8]:
# Define a function to check if the "没有更多交流了~" message is displayed
def no_more_messages():
    try:
        # Check if the element is present on the page
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'msg-notext'))
        )
        # Check if the element contains the specified text
        return "没有更多交流了~" in element.text
    except:
        return False

# Define a function to click the "查看更多交流" button
def click_more_button():
    try:
        # Click the "查看更多交流" button
        more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'msg-more-link-text'))
        )
        more_button.click()
        time.sleep(8)
        return True
    except:
        return False

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
link="https://passport.haodf.com/nusercenter/showlogin"
driver.get(link)

/var/folders/r7/p1s2dy_d55zbvx8s9j4nd_x00000gn/T/ipykernel_12386/2834877089.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
/var/folders/r7/p1s2dy_d55zbvx8s9j4nd_x00000gn/T/ipykernel_12386/2834877089.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)


In [36]:
# manually login in

In [10]:
l_num = np.load(f'{outpath}/num.npy')
print('数字文件名列表长度：',len(l_num))

数字文件名列表长度： 300


In [11]:
#l_num = np.array([])

In [12]:
start = input()

 663


In [13]:
for i in range(int(start),int(start)+100):
    link = l[i]
    num = link.split('/')[-1].split('.')[0]
    l_num = np.append(l_num, num)
    
    print("正在下载第",i,'个网页链接。')

    driver.get(link)
    # Keep clicking the button until "没有更多交流了~" is displayed
    f = 0
    while not no_more_messages():
        click_more_button()
        f += 1
        if f > 5:
            print(link,' failed.')
            break
            
    html_content = driver.page_source
    
    with open(f'{outpath}/{num}.html', 'w', encoding='utf-16') as file:
        file.write(html_content)

print(start,'-',int(start)+100, 'are saved.')

正在下载第 663 个网页链接。
正在下载第 664 个网页链接。
正在下载第 665 个网页链接。
正在下载第 666 个网页链接。
正在下载第 667 个网页链接。
正在下载第 668 个网页链接。
正在下载第 669 个网页链接。
正在下载第 670 个网页链接。
正在下载第 671 个网页链接。
正在下载第 672 个网页链接。
正在下载第 673 个网页链接。
正在下载第 674 个网页链接。
正在下载第 675 个网页链接。
正在下载第 676 个网页链接。
正在下载第 677 个网页链接。
https://www.haodf.com/bingcheng/8897283756.html  failed.
正在下载第 678 个网页链接。
正在下载第 679 个网页链接。
正在下载第 680 个网页链接。
正在下载第 681 个网页链接。
正在下载第 682 个网页链接。
正在下载第 683 个网页链接。
正在下载第 684 个网页链接。
正在下载第 685 个网页链接。
正在下载第 686 个网页链接。
正在下载第 687 个网页链接。
正在下载第 688 个网页链接。
正在下载第 689 个网页链接。
正在下载第 690 个网页链接。
正在下载第 691 个网页链接。
正在下载第 692 个网页链接。
正在下载第 693 个网页链接。
正在下载第 694 个网页链接。
正在下载第 695 个网页链接。
正在下载第 696 个网页链接。
正在下载第 697 个网页链接。
正在下载第 698 个网页链接。
正在下载第 699 个网页链接。
正在下载第 700 个网页链接。
正在下载第 701 个网页链接。
正在下载第 702 个网页链接。
正在下载第 703 个网页链接。
正在下载第 704 个网页链接。
正在下载第 705 个网页链接。
正在下载第 706 个网页链接。
正在下载第 707 个网页链接。
正在下载第 708 个网页链接。
正在下载第 709 个网页链接。
正在下载第 710 个网页链接。
正在下载第 711 个网页链接。
正在下载第 712 个网页链接。
正在下载第 713 个网页链接。
https://www.haodf.com/bingcheng/8897283508.html  failed.
正在下载第 714 个网页链接。
正在

In [18]:
driver.quit()

## 提取信息

In [19]:
# 你的目录路径
directory_path = '../Data/webpage'

# 获取HTML文件列表
html_files_list = get_html_files(directory_path)

np.save(f'{outpath}/num.npy', html_files_list)

In [20]:
l_num = np.load(f'{outpath}/num.npy')
print(len(l_num))

750


In [21]:
l_num_flaw = []

In [26]:
#l_num_flaw = np.load(f'{outpath}/num_flaw.npy')

In [27]:
#print(l_num_flaw)

['8897283364' '8897283508' '8897283756' '8897285638' '8897286429'
 '8897287010' '8897287389' '8897287926' '8897288332']


### info

In [23]:
def insert_cat(df,cat):
    df.insert(0, 'category', cat)
    df.insert(1, 'num', range(1, len(df) + 1))
    return df

In [24]:
#INFO
def meta(soup):
    # Extract meta tags with specified attributes
    meta_tags = soup.find_all('meta', {'name': 'description'})
    meta_tags += soup.find_all('meta', {'name': 'keywords'})
    meta_tags += soup.find_all('meta', {'name': 'mobile-agent'})
    meta_tags += soup.find_all('meta', {'property': 'og:url'})
    meta_tags += soup.find_all('meta', {'property': 'og:type'})
    meta_tags += soup.find_all('meta', {'property': 'og:title'})
    meta_tags += soup.find_all('meta', {'property': 'og:release_date'})
    meta_tags += soup.find_all('meta', {'property': 'og:description'})
    
    # Create a DataFrame from the extracted meta tags
    df_meta = pd.DataFrame([(tag.attrs.get('name') or tag.attrs.get('property'), tag.attrs.get('content')) for tag in meta_tags],
                       columns=['attribute', 'value'])
    df_meta = insert_cat(df_meta,'meta')
    return df_meta

def disease_info(soup):
    # Extracting information from the HTML (Disease Info)
    disease_info = soup.select_one('.diseaseinfo')
    info_items = disease_info.find_all(['span', 'br'])
    
    data_disease_info = {'attribute': [], 'value': []}
    current_attribute = None
    
    for item in info_items:
        if item.name == 'span' and 'info3-title' in item.get('class', []):
            current_attribute = item.text.strip()
        elif item.name == 'span' and 'info3-value' in item.get('class', []):
            # If the attribute already exists, concatenate the new value with a newline
            if current_attribute in data_disease_info['attribute']:
                idx = data_disease_info['attribute'].index(current_attribute)
                data_disease_info['value'][idx] += '\n' + item.text.strip()
            else:
                data_disease_info['attribute'].append(current_attribute)
                data_disease_info['value'].append(item.text.strip())
    
    # Creating a DataFrame for Disease Info
    df_disease_info = pd.DataFrame(data_disease_info)
    df_disease_info = insert_cat(df_disease_info,'disease_info')
    return df_disease_info

def suggestions(soup):
    # Extracting information from the HTML (Suggestions)
    suggestions_sections = soup.select('.suggestions-text')
    
    data_suggestions = {'attribute': [], 'value': []}
    current_attribute_suggestions = None
    
    for suggestions_section in suggestions_sections:
        curr_head = suggestions_section.select_one('.curr-head')
        if curr_head:
            current_attribute_suggestions = curr_head.text.strip()
        suggestions_text_value = suggestions_section.select_one('.suggestions-text-value')
        if suggestions_text_value:
            # If the attribute already exists, concatenate the new value with a newline
            if current_attribute_suggestions in data_suggestions['attribute']:
                idx = data_suggestions['attribute'].index(current_attribute_suggestions)
                data_suggestions['value'][idx] += '\n' + suggestions_text_value.text.strip()
            else:
                data_suggestions['attribute'].append(current_attribute_suggestions)
                data_suggestions['value'].append(suggestions_text_value.text.strip())
    
    # Creating a DataFrame for Suggestions
    df_suggestions = pd.DataFrame(data_suggestions)
    df_suggestions = insert_cat(df_suggestions,'suggestions')
    return df_suggestions

def doctor_info(soup):
    # Extract relevant information (Doctor Info)
    info_text_name = soup.find('span', class_='info-text-name').text.strip()
    info_text_grade = soup.find('span', class_='info-text-grade').text.strip()
    hospital = soup.find('a', class_='hospital').text.strip()
    faculty = soup.find('a', class_='faculty').text.strip()
    
    # Creating a DataFrame for Doctor Info
    df_doctor_info = pd.DataFrame({'attribute': ['info-text-name', 'info-text-grade', 'hospital', 'faculty'],
                                'value': [info_text_name, info_text_grade, hospital, faculty]})
    
        # Extract additional relevant information
    add_info = soup.find_all('div', class_='item-title')
    add_info_value = soup.find_all('div', class_='item-detail')
    
    # Creating a DataFrame for Additional Info
    df_additional = pd.DataFrame({'attribute': [add_info[0].text.strip(), add_info[1].text.strip()],
                              'value': [add_info_value[0].text.strip(), add_info_value[1].text.strip()]})
    
    
    # Extract the HTML content as a string
    html_string = str(soup)
    # Define the regex pattern
    pattern = r'href="(https://www\.haodf\.com/doctor/\d+\.html)"'
    # Use re.findall to extract the URL
    match = re.search(pattern, html_string)
    url = match.group(1)
        
    # Define the regex pattern to extract the doc_id
    doc_id_pattern = r'/doctor/(\d+)\.html'
    match = re.search(doc_id_pattern, url)
    
    # Extract doc_id if a match is found
    doc_id = match.group(1) if match else None
    
    # Create a DataFrame
    data = {'attribute': ['url', 'doc_id'],
            'value': [url, doc_id]}
    
    df_docid = pd.DataFrame(data)

    
    df_doctor_info = pd.concat([df_doctor_info, df_additional, df_docid],ignore_index=True)
    
    df_doctor_info = insert_cat(df_doctor_info,'doctor_info')
    
    
    return df_doctor_info

    
def patient(soup):
    # Extract relevant information (Patient Info)
    patient_info = soup.find('div', class_='patient-card-wrap')
    
    # Check if the element is found
    if patient_info:
        patient_name = patient_info.find('p', class_='patient-card-info').text.strip()
    
        # Creating a DataFrame for Patient Info
        df_patient = pd.DataFrame({'attribute': ['就诊患者'],
                                   'value': [patient_name]})
    
    else:
        print("Patient information not found.")
        df_patient = pd.DataFrame(columns=['attribute', 'value'])
    df_patient = insert_cat(df_patient,'patient')
    return df_patient

In [25]:
for i in range(0,len(l_num)):
    html_file_path = f'{outpath}/{l_num[i]}.html'
    # Read the content of the HTML file
    with open(html_file_path, 'r', encoding='utf-16') as file:
               html_content = file.read()
               
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        df_meta = meta(soup)
    
        df_disease_info = disease_info(soup)
    
        df_suggestions = suggestions(soup)
    
        df_doctor_info =  doctor_info(soup)
        
        df_patient = patient(soup)
    
    
        # Combine all DataFrames into one coherent DataFrame
        df_combined = pd.concat([df_meta, df_disease_info, df_suggestions, df_doctor_info, df_patient],
                           ignore_index=True)
        # Print the resulting DataFrame
        #print(df_combined)
        save_dataframe(df_combined, l_num[i], outpath, "info")
    except: 
        l_num_flaw.append(l_num[i])
    if i%50==0:
        print(i,'...')
print(str(len(l_num)-len(np.unique(l_num_flaw))), 'are saved, ',len(np.unique(l_num_flaw)),"failed.")
np.save(f'{outpath}/num_flaw.npy', np.unique(l_num_flaw))

0 ...
50 ...
100 ...
150 ...
200 ...
250 ...
300 ...
350 ...
400 ...
450 ...
500 ...
550 ...
600 ...
650 ...
700 ...
741 are saved,  9 failed.


In [28]:
l_num = [item for item in l_num if item not in l_num_flaw]

In [29]:
np.save(f'{outpath}/num.npy', l_num)

In [30]:
len(l_num)

741

### dialogue

In [31]:
l_num = np.load(f'{outpath}/num.npy')
print(len(l_num))

741


In [32]:
def dialogue(soup):
    messages = soup.find_all('div', class_='msg-item')
    #print(len(messages))
    
    data = []
    for message in messages:
        time_element = message.find('div', class_='msg-time')
        time = time_element.text.strip() if time_element else "Unknown Time"
    
        if 'item-left' in message['class'] or 'item-right' in message['class']:
            sender_span = message.find('span', class_='content-name')
            sender = sender_span.text.strip() if sender_span else "System"  # Default to "System" if sender is not found
    
            # Check for different message types
            content_span = message.find('span', class_='content-text')
            if content_span:
                content = content_span.text.strip()
            else:
                # Handle other types of messages (e.g., life advice, diagnosis, system notices)
                content_com_title = message.find('span', class_='content-com-title')
                content_com_text = message.find('span', class_='content-com-text')
                if content_com_title and content_com_text:
                    content = f'{content_com_title.text.strip()}: {content_com_text.text.strip()}'
                elif content_com_title:
                    content = content_com_title.text.strip()
                elif content_com_text:
                    content = content_com_text.text.strip()
                else:
                    content = "Unknown message type"
    
            data.append({'Time': time, 'Sender': sender, 'Content': content})
    
    df = pd.DataFrame(data)
    return df

In [33]:
#q = [9,12,14,30,32,35,44,55,57,60,71,95,102,107,113,119,133,139,163,170,181,187,194,197]

In [34]:
l_num_flaw = np.load(f'{outpath}/num_flaw.npy')

In [35]:
for i in range(len(l_num)):
    html_file_path = f'{outpath}/{l_num[i]}.html'
    # Read the content of the HTML file
    with open(html_file_path, 'r', encoding='utf-16') as file:
        html_content = file.read()
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        df = dialogue(soup)
    
        save_dataframe(df, l_num[i], outpath,"dialogue")
    except:
        l_num_flaw.append(l_num[i])
    if i%50==0:
        print(i,'...')
print(str(len(l_num)-len(np.unique(l_num_flaw))), 'are saved, ',len(np.unique(l_num_flaw)),"failed.")
np.save(f'{outpath}/num_flaw.npy', np.unique(l_num_flaw))

0 ...
50 ...
100 ...
150 ...
200 ...
250 ...
300 ...
350 ...
400 ...
450 ...
500 ...
550 ...
600 ...
650 ...
700 ...
732 are saved,  9 failed.


In [38]:
l_num_flaw = np.load(f'{outpath}/num_flaw.npy')

In [39]:
print(l_num_flaw)

['8897283364' '8897283508' '8897283756' '8897285638' '8897286429'
 '8897287010' '8897287389' '8897287926' '8897288332']
